# Imports

In [1]:
import multiprocessing
import os
import time
from UTILITY_quickstart import *

In [2]:
max_workers = len(os.sched_getaffinity(0))
print(f"Available cores: {max_workers}")

importedDefaultSettings = loadConfig(f'setLattice_configs/2025-02-25_oneBunch_baseline.yml')

Available cores: 256


### Make Synthetic Distribution

In [3]:
# From E. Cropp RedPill Helper_functions.py #
#############################################
def Gaussian_Dist_Maker(n,mu,sigma,lSig,rSig):
    """
    This function returns a truncated gaussian distribution of quasi-random particles.  This uses the Halton series
    
    Argument:
    n -- int number of particles
    mu -- float: center of distribution/mean
    sigma -- float: std of distribution
    lSig -- float number of sigma at which to truncate Gaussian left
    rSig -- float number of sigma at which to truncate Gaussian right
    """
    # Check inputs
    try: n = int(n)
    except: raise ValueError("n is not an int!")
    
    try: mu = float(mu)
    except: raise ValueError("mu is not a float!")
    
    try: sigma = float(sigma)
    except: raise ValueError("sigma is not a float!")
    
    try: lSig = float(lSig)
    except: raise ValueError("lSig is not a float!")
    
    try: rSig = float(rSig)
    except: raise ValueError("rSig is not a float!")
    
    
    # get and shuffle n samples from halton series
    h=scipy.stats.qmc.Halton(1)
    X0=h.random(n=n)
    np.random.shuffle(X0)
    
    # Make these into Gaussian and return
    X0=X0*(1-(1-scipy.stats.norm.cdf(lSig))-(1-scipy.stats.norm.cdf(rSig)))
    X0=X0+(1-scipy.stats.norm.cdf(lSig))
    GaussDist = mu + np.sqrt(2)*sigma*scipy.special.erfinv(2*X0-1)
    return np.squeeze(GaussDist)

### Initialize nominal parameters and jitter margins

In [4]:
# I think L0B, L1, L2, L3 amplitudes can be jittered by adding/subtracting to <Stage>EnergyOffset.
nominal_charge             = 1600 # pC
nominal_gun_theta0_deg     = 29.3-90
nominal_gun_rf_field_scale = 119/2.44885*1e6
nominal_L0A_Phase          = 29
nominal_L0A_Amp            = 30e6
nominal_L0B_Phase          = importedDefaultSettings['L0BPhaseSet']
nominal_L0BF_Amp           = 5.95e7
nominal_L1_Phase           = importedDefaultSettings['L1PhaseSet']
nominal_L1_Amp             = 0.335e9 - 0.125e9
nominal_L2_Phase           = importedDefaultSettings['L2PhaseSet']
nominal_L2_Amp             = 4.5e9 - 0.335e9
nominal_L3_Phase           = importedDefaultSettings['L3PhaseSet']
nominal_L3_Amp             = 10.0e9 - 4.5e9

# values obtained from `2024 run` column of https://docs.google.com/spreadsheets/d/1xeCUImz5uFSq6QA3wV91dG38s-8cyVXQMGw9hjPKa6M/edit?usp=sharing
charge_jitter_percent  = 2.3
gun_Phase_jitter       = 0.15
gun_Amp_jitter_percent = 0.25
L0A_Phase_jitter       = 0.1 # L0APhaseOffset kwarg passed directly into `initializeTao`. Note that it's an *offset*.
L0A_Amp_jitter_percent = 0.06
L0B_Phase_jitter       = 0.1
L0B_Amp_jitter_percent = 0.5
L1A_Phase_jitter       = 0.7
L1A_Amp_jitter_percent = 0.6
L1B_Phase_jitter       = 0.5
L1B_Amp_jitter_percent = 0.7
L2_Phase_jitter        = 0.4
L2_Amp_jitter_percent  = 0.3
L3_Phase_jitter        = 0.4
L3_Amp_jitter_percent  = 0.3

L_str = 'L2'

cal_data = {
    f'{L_str}PhaseSet':[nominal_L2_Phase - L2_Phase_jitter, nominal_L2_Phase + L2_Phase_jitter],
    f'L1PhaseSet':[nominal_L1_Phase - L1A_Phase_jitter, nominal_L1_Phase + L1A_Phase_jitter],
}

# Beam output locations
locations = ['L0AFEND','ENDINJ','BEGL1F','ENDL1F','BC11CEND','ENDL2F','ENDBC14_2','ENDL3F_2','BEGFF20','ENDFF20','PENT']

### Make evaluation points

In [5]:
N = 10 # max_workers
cutoff_sigma = 3

# Quasi-random Gaussian
points = {}
for key in cal_data.keys():
    mu = np.mean(np.array(cal_data[key]))
    sigma = np.ptp(np.array(cal_data[key]))/(2*cutoff_sigma)
    dist = Gaussian_Dist_Maker(N,mu,sigma,cutoff_sigma,cutoff_sigma)
    points[key] = dist
# points = pd.DataFrame(points)

# turn dict of arrays into array of dicts
tasks = [
    {key: points[key][i] for key in cal_data.keys()}
    for i in range(N)
]

# Parallelizable jitter function

In [6]:
path_conda = '/global/homes/m/maxvarv/miniforge3/envs/bmad/bin/'

output_path = f'/pscratch/sd/m/maxvarv/Jitter_2025_02_25/{L_str}_Phase'
os.makedirs(output_path, exist_ok=True)

def worker(overrides):
    start_time = time.time()
    
    importedDefaultSettings = loadConfig(f'setLattice_configs/2025-02-25_oneBunch_baseline.yml')

    tao, unique_ID = initializeTao(
            inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
            GFILESuffix = f'2024-10-22_distgen_onebunch.yaml',
            csrTF = True,
            runImpactTF = False,
            # impactGridCount = 32,          # unused when `runImpactTF = False`
            # numMacroParticles = 5 * 32**3, # unused when `runImpactTF = False` (still technically used if uncommented, but we want to use the h5 input file to define this)
            # solenoidTValue = -0.41, # uncomment if explicitly running impact-T!
            # impactChargepC = point['ChargepC'], # adjust bunch charge (when jittering in impact-T),
            command = path_conda + 'ImpactTexe',    
            command_mpi = path_conda + 'ImpactTexe-mpi',
            mpi_run = '/global/u1/m/maxvarv/miniforge3/envs/bmad/bin/mpirun --map-by :OVERSUBSCRIBE -n {nproc} {command_mpi}',
            scratchPath = output_path,
            randomizeFileNames = True,
    	)

    activeSettings = importedDefaultSettings | overrides

    try:
        setLattice(tao, **activeSettings)
        disableAutoQuadEnergyCompensation(tao)
        
        trackBeam(tao, **activeSettings)
    
        print(unique_ID, overrides["L2PhaseSet"], overrides["L1PhaseSet"])
        result = {'Unique ID': unique_ID}
        result.update(overrides)
        
        # make directory if it doesn't already exist
        os.makedirs(f'{output_path}/{unique_ID}', exist_ok=True)
        
        for location in locations:
            P = getBeamAtElement(tao, location)
            
            # write to output_path
            P.write(f'{output_path}/{unique_ID}/{location}.h5')
    finally:
        tao.close_subprocess()
        print(f'Run {ID} elapsed time: {(time.time() - start_time) / 60:.1f} minutes')
        
    return result

In [7]:
with multiprocessing.Pool(N) as pool:
    results = pool.map(worker, tasks)

jitter_numbers_df = pd.DataFrame(results)
jitter_numbers_df.to_csv(f'{output_path}/jitter_numbers.csv')
print(jitter_numbers)

for res in results:
    print(res)

Environment set to: Environment set to: Environment set to:   Environment set to:  Environment set to: Environment set to: /global/u1/m/maxvarv/FACET2-Bmad-PyTao/global/u1/m/maxvarv/FACET2-Bmad-PyTao/global/u1/m/maxvarv/FACET2-Bmad-PyTao 
 
 
/global/u1/m/maxvarv/FACET2-Bmad-PyTao/global/u1/m/maxvarv/FACET2-Bmad-PyTao/global/u1/m/maxvarv/FACET2-Bmad-PyTao


Environment set to: Environment set to: Environment set to: Environment set to:   /global/u1/m/maxvarv/FACET2-Bmad-PyTao  
/global/u1/m/maxvarv/FACET2-Bmad-PyTao/global/u1/m/maxvarv/FACET2-Bmad-PyTao/global/u1/m/maxvarv/FACET2-Bmad-PyTao


Tracking to end
Tracking to endTracking to end

Tracking to end
Tracking to end
Tracking to end
Tracking to end
Tracking to endTracking to end

Tracking to end
CSR on
Overwriting lattice with setLattice() defaultsCSR on
No defaults file provided to setLattice(). Using setLattice_configs/defaults.ymlCSR on

Overwriting lattice with setLattice() defaults

Overwriting lattice with setLattice() defaul